In [ ]:
import joblib
import pandas as pd
import os

CODESPACES = True

if CODESPACES:
    os.chdir('/workspaces/fraud_mlops')
else:
    os.chdir('/Users/michael.hodge/GitProjects/fraud_mlops')
print(os.getcwd())

from scripts.utils.data_pipeline import data_pipeline_predict

df = pd.read_csv('./data/synthetic_fraud_dataset.csv')
first_row = df.iloc[:1]
first_row

/workspaces/fraud_mlops


,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,...,7,437.63,3,Amex,65,883.17,Biometric,0.8494,0,0


In [2]:
scaler = joblib.load('./artefacts/scaler.save')
encoder = joblib.load('./artefacts/encoder.save')
selector = joblib.load('./artefacts/selector.save')

In [3]:
input_data = data_pipeline_predict(first_row, scaler, encoder, selector)
input_data

/workspaces/fraud_mlops/scripts/utils/data_pipeline.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_timestamp'] = pd.to_datetime(df['transaction_timestamp'])
/workspaces/fraud_mlops/scripts/utils/data_pipeline.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = df['transaction_timestamp'].dt.day_name()
/workspaces/fraud_mlops/scripts/utils/data_pipeline.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

array([[-1.35299369,  1.49251806, -1.22649279, -1.41571843, -1.41472898,
        -0.23044883, -0.12151956,  0.70851655, -1.34935421,  1.20978316,
        -1.4948994 ,  1.0900955 , -0.24484035, -0.57713033, -2.42465291]])

In [4]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow.sklearn

RUN_ID = 'ac26a369dd364795bc5f4463d4e2997a'

logged_model = f'runs:/{RUN_ID}/XGBClassifier'

# Load model as a PyFuncModel.
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(input_data)
y_pred_prob = loaded_model.predict_proba(input_data)

prob_perc = round(float(max(max(y_pred_prob)))*100,2)

if y_pred[0] == 1:
    print(f"This transaction could be FRAUD with a probability score of {prob_perc}%")
else:
    print(f"This transaction is likely to be legitimate with a probability score of {prob_perc}%")


This transaction could be FRAUD with a probability score of 79.41%
